In [1]:
# -*- coding: utf-8 -*-
import os
import mwclient
import json
import time
import re
import requests
from bs4 import BeautifulSoup
import lxml
from lxml.html import fromstring

import xlwt
import xlrd
import warnings
warnings.filterwarnings('ignore')

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import pandas as pd

In [2]:
url_nbh = "https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Jerusalem"
webdata_nbh = requests.get(url_nbh)
soup_nbh = BeautifulSoup(webdata_nbh.text, "html.parser")
# neighborhoods chose from 'Pages in category "Neighbourhoods of Jerusalem"'
list_nbh = soup_nbh.find_all('div', attrs={'class':'mw-category-group'})

In [3]:
url_lop = "https://en.wikipedia.org/wiki/List_of_places_in_Jerusalem"
webdata_lop = requests.get(url_lop)
root = fromstring(webdata_lop.content)
table = root.xpath(".//h2[span = 'Neighborhoods']/following-sibling::table")[0]
row = table.xpath(".//tr")[0]
list_lop = [cell.xpath(".//li") for cell in row.xpath(".//td")]

In [4]:
links, titles = [], []
for i in range(12, 34):
    temp_list = list_nbh[i].find_all('li')
    for item in temp_list:
        link = item.find('a')['href']
        link = 'https://en.wikipedia.org/'+str(link)
        title = item.find('a')['title']
        links.append(link)
        titles.append(title)

In [5]:
links2, titles2 = [], []
for sub_list in list_lop:
    for item in sub_list:
        link = item[0].get('href')
        link = 'https://en.wikipedia.org/'+str(link)
        title = item[0].get('title')
        links2.append(link)
        titles2.append(title)

In [6]:
workbook = xlwt.Workbook(r'wiki_nbh_info.xls')
sheet = workbook.add_sheet('wiki_nbh', cell_overwrite_ok=True)
sheet2 = workbook.add_sheet('wiki_lop', cell_overwrite_ok=True)
for i in range(len(links)):
    sheet.write(i, 0, titles[i])
    sheet.write(i, 1, links[i])
    sheet.write(i, 2, 'Category:Neighbourhoods_of_Jerusalem')
for i in range(len(links2)):
    sheet2.write(i, 0, titles2[i])
    sheet2.write(i, 1, links2[i])
    sheet2.write(i, 2, 'List_of_places_in_Jerusalem: Neighborhoods')
workbook.save(r"wiki_nbh_info.xls")

In [7]:
wb_rd = xlrd.open_workbook(r'jerusalem_data.xls')
sheet_rd = wb_rd.sheet_by_index(0)
titles3 = sheet_rd.col_values(0, start_rowx=1)

In [8]:
nbh_df = pd.DataFrame(data=[titles, links]).T
nbh_df.columns = ['titles','links']
nbh_df['source'] = 'Category:Neighbourhoods_of_Jerusalem'

In [9]:
lop_df = pd.DataFrame(data=[titles2, links2]).T
lop_df.columns=['titles','links']
lop_df['source'] = 'List_of_places_in_Jerusalem'

In [10]:
wiki_df = pd.merge(nbh_df, lop_df, how='outer', suffixes=["1", "2"],
         left_on=['titles', 'links'], right_on=['titles', 'links'])

In [11]:
wiki_df

,titles,links,source1,source2
0,Abu Tor,https://en.wikipedia.org//wiki/Abu_Tor,Category:Neighbourhoods_of_Jerusalem,List_of_places_in_Jerusalem
1,"American Colony, Jerusalem",https://en.wikipedia.org//wiki/American_Colony...,Category:Neighbourhoods_of_Jerusalem,NaN
2,Armenian Quarter,https://en.wikipedia.org//wiki/Armenian_Quarter,Category:Neighbourhoods_of_Jerusalem,List_of_places_in_Jerusalem
3,Arnona,https://en.wikipedia.org//wiki/Arnona,Category:Neighbourhoods_of_Jerusalem,List_of_places_in_Jerusalem
4,Arzei HaBira,https://en.wikipedia.org//wiki/Arzei_HaBira,Category:Neighbourhoods_of_Jerusalem,NaN
...,...,...,...,...
174,"San Simon, Jerusalem","https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,List_of_places_in_Jerusalem
175,Sanhedria Murchevet,https://en.wikipedia.org//wiki/Sanhedria_Murch...,NaN,List_of_places_in_Jerusalem
176,Unsdorf,https://en.wikipedia.org//wiki/Unsdorf,NaN,List_of_places_in_Jerusalem
177,Zichron Moshe,https://en.wikipedia.org//wiki/Zichron_Moshe,NaN,List_of_places_in_Jerusalem


In [23]:
book_df = pd.read_excel(io=r'jerusalem_data.xls')

In [24]:
sub_df = pd.DataFrame(data = [process.extract(str(item), wiki_df['titles'], limit=2, 
        scorer=fuzz.partial_ratio) for item in book_df['Neighborhood']])
sub_df.columns = ['substitute1','substitute2']

In [25]:
sub_df

,substitute1,substitute2
0,"(Mishkenot Sha'ananim, 100, 90)","(Ir Ganim, 62, 52)"
1,"(Mahane Israel, 85, 82)","(Malha, 80, 84)"
2,"(Nahalat Shiv'a, 93, 98)","(Nahalat Shiva, 85, 167)"
3,"(Kerem Avraham, 77, 62)","(Tel Arza, 75, 140)"
4,"(Beit David, 90, 14)","(Bezetha, 57, 21)"
...,...,...
161,"(Romema, 100, 123)","(Romema, Jerusalem, 100, 173)"
162,"(Armenian Quarter, 57, 2)","(Beit Hanina, 57, 16)"
163,"(Mamilla, 71, 85)","(Musrara, 71, 95)"
164,"(Romema, 60, 123)","(Musrara, 57, 95)"


In [27]:
pd.concat([book_df, sub_df], axis=1)

,Neighborhood,Year founded,Inhabitants,Initiative,Remarks,Type of neighborhood,Unnamed: 6,substitute1,substitute2
0,Mishkenot Sha'ananim,1855-1860,"Ashkenazi, Sephardi",Philanthropic,NaN,Jewish and mixed neighborhoods,Development of the Jerusalem neighborhoods out...,"(Mishkenot Sha'ananim, 100, 90)","(Ir Ganim, 62, 52)"
1,Mahaneh Yisra’el,1866,Maghrebian,Private,NaN,Jewish and mixed neighborhoods,NaN,"(Mahane Israel, 85, 82)","(Malha, 80, 84)"
2,Nanalat Shiv’a,1869,"Ashkenazi, Sephardi",Private,NaN,Jewish and mixed neighborhoods,NaN,"(Nahalat Shiv'a, 93, 98)","(Nahalat Shiva, 85, 167)"
3,Georgian Houses (Eshel Avraham),1872-1876,Georgian,Building society and private,NaN,Jewish and mixed neighborhoods,NaN,"(Kerem Avraham, 77, 62)","(Tel Arza, 75, 140)"
4,Bet David (Yanover Houses),1873,Ashkenazi,Philanthropic,NaN,Jewish and mixed neighborhoods,NaN,"(Beit David, 90, 14)","(Bezetha, 57, 21)"
...,...,...,...,...,...,...,...,...,...
161,Romema,1922,"Ashkenazi and Oriental Jews, Muslims",Private,Named after Psalms 118:16,Mixed neighborhoods,NaN,"(Romema, 100, 123)","(Romema, Jerusalem, 100, 173)"
162,Dajania*,Early Mandate,"Muslims, Christians",Private,NaN,Mixed neighborhoods,NaN,"(Armenian Quarter, 57, 2)","(Beit Hanina, 57, 16)"
163,al-Musallaba,Early Mandate,Mixed (mainly Jews),NaN,Part of Rehavia?,Mixed neighborhoods,NaN,"(Mamilla, 71, 85)","(Musrara, 71, 95)"
164,St. Louis Road*,Early Mandate,"Jews, Muslims, Christians",NaN,NaN,Mixed neighborhoods,NaN,"(Romema, 60, 123)","(Musrara, 57, 95)"
